In [15]:
import numpy as np
import xarray as xr
from pathlib import Path
import act
import matplotlib.pyplot as plt
import matplotlib
import glob
%matplotlib inline

print('Reese')

Reese


# Opening a single pblht .cdf file from sgp

Checking available data varibles. We are going to want to filter by the boundary layer regime type (0-neutral)

Basetime is an available data variable, thus we should be able to drop everything that isnt basetime, pbl_regime_type, and pbl_height.

After concatenating a month of .cdf into a single dataset, we should be able to filter (regime=neutral) and store the filtered dataset into a new .cdf file.

In [16]:
ds1 = xr.open_dataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.20190430.233200.cdf")
ds1
ds2 = xr.open_dataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.20190430.233200.cdf")

<xarray.Dataset>
Dimensions:                             (height_ss: 188, layer: 5, time: 4357)
Coordinates:
  * time                                (time) datetime64[ns] 2019-04-30T23:3...
  * height_ss                           (height_ss) float32 314.8 ... 2.207e+04
  * layer                               (layer) int32 1 2 3 4 5
Data variables: (12/33)
    base_time                           datetime64[ns] ...
    time_offset                         (time) datetime64[ns] ...
    atm_pres                            (time) float32 ...
    air_temp                            (time) float32 ...
    wspd                                (time) float32 ...
    rh                                  (time) float32 ...
    ...                                  ...
    delta_theta_max                     (layer) float32 ...
    level_1_liu_liang                   float32 ...
    level_2_liu_liang                   float32 ...
    lat                                 float32 ...
    lon                                 float32 ...
    alt                                 float32 ...
Attributes: (12/16)
    command_line:                   idl -D 0 -R -n pblhtsonde -s sgp -f C1 -d...
    process_version:                vap-pblhtsonde-0.8-0.el6
    dod_version:                    pblhtsonde1mcfarl-c1-2.0
    site_id:                        sgp
    platform_id:                    pblhtsonde1mcfarl
    facility_id:                    C1: Lamont, Oklahoma
    ...                             ...
    overshoot_threshold:            4
    reference_1:                    Heffter, J. L., 1980: Transport layer dep...
    reference_2:                    Liu, S. and X. Z. Liang, 2010: Observed D...
    input_datastreams:              sgpsondewnpnC1.b1 : 10.12 : 20190430.0231...
    input_datastreams_description:  A string consisting of the datastream(s),...
    history:                        created by user dsmgr on machine ruby at ...

In [13]:
ds1['pbl_regime_type_liu_liang']

<xarray.DataArray 'pbl_regime_type_liu_liang' ()>
array(0.)
Attributes:
    long_name:           Planetary boundary layer regime type determined by L...
    units:               unitless
    flag_values:         0, 1, 2
    flag_meanings:       neutral_boundary_layer stable_boundary_layer convect...
    flag_0_description:  Neutral boundary layer
    flag_1_description:  Stable boundary layer
    flag_2_description:  Convective boundary layer

In [31]:
#dsa = xr.concat([xr.open_dataset(f) for f in glob.glob(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\*.cdf")], dim='time')
filename = str(Path(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf"))
filename


'C:\\Users\\Reese\\Desktop\\AOS Proj Data\\pblht\\sgppblhtsonde1mcfarlC1.c1.*.cdf'

In [34]:
met_ds = act.io.armfiles.read_netcdf(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf")
met_ds

TypeError: open_dataset() got an unexpected keyword argument 'combine_attrs'

In [37]:
ds = xr.open_mfdataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf", concat_dim='time')

ValueError: Coordinate variable height_ss is neither monotonically increasing nor monotonically decreasing on all datasets

# Possible Option -- 

Might be able to open a new .cdf, loop through the directory containing month's worth of sonde data, check regime type in each iteration and if neutral, pull the pbl height and sonde time to write to new .cd

In [ ]:
import os

directory = r"C:\Users\Reese\Desktop\AOS Proj Data\pblht"

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)